# GLUT1 Analysis - Main Notebook

### 1. Preparing the data
Folder structure:
+ *Main Folder*
    + *Rawdata*
    + *Templates*
    + Animal_Scan_IDs.xlsx
    
    
- In the *Rawdata* folder each animal has a subfolder containing all the raw scans (simply the folders given by ParaVision)
- *Templates* folder contains templates for coregistration for each MR sequence (just pick one of the animals in the group). See examples.
- *Excel* file contains all animal ID's and their corresponding scan numbers for each modality (see example file).

In [1]:
import os
import subprocess
import numpy as np
import pandas as pd
import MRIhelperfunctions as h
import MRIhelperfunctions
from MRIhelperfunctions import Settings
from imp import importlib
import fnmatch
import shutil
import papermill as pm
import glob


In [192]:
importlib.reload(h)
import MRIhelperfunctions as h
from MRIhelperfunctions import Settings

## Folder settings
Adjust these settings once for each study (namely location of rawdata folder and name of excel info file). These are constant across all animals and modalities

In [2]:
raw_folder = '/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata'   #<-- change this
main_folder = os.path.dirname(raw_folder)  # automatically uses one folder above rawdata folder as "main folder".

excel_filename = 'Animal_Scan_IDs.xlsx'  #<-- change this. Must be placed inside main folder


analysis_folder = os.path.join(main_folder,'Analysis')
notebookfolder = '/BigHDD/MR_DATA/GithubRepositories/CalciumfMRI/fMRI_notebooks'

folders = {}
folders.update({'main':main_folder, 'raw':raw_folder, 'analysis':analysis_folder, 'notebooks':notebookfolder, 'excel':excel_filename})
# folders = Settings(main = main_folder, raw = raw_folder, analysis = analysis_folder, notebooks = notebookfolder, excel = excel_filename)

# Check in output below if Excel file is correctly recognized
info = h.getinfo(folders)
info

,glut1_male1,glut1_male2,glut1_f10wks1,glut1_f10wks2,glut1_f10wks3,glut1_f10wks4,glut1_f10wks5,glut1_f10wks6,glut1_f8wks1,glut1_f8wks2,glut1_f8wks3,glut1_f8wks4,glut1_f8wks5,glut1_f8wks6,glut1_f8wks7,glut1_f6wks2,glut1_f6wks1
scan,,,,,,,,,,,,,,,,,
genotype,1,1,1,2,1,2,2,1,1,1,2,2,2,2,1,1,1
Gd DCE,6,8,6,5,5,8,5,5,7,5,6,6,8,8,6,5,5
Angio pre,8,9,7,6,6,9,6,6,8,6,9,8,9,9,8,6,6
Angio post,12,12,11,9,9,13,10,11,12,11,12,12,16,13,12,9,10
Endorem DCE,11,11,10,8,8,12,9,10,10,10,11,11,15,12,11,8,9
Diamox,15,14,13,11,13,15,12,13,15,13,16,17,20,16,14,11,13
folder,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018060...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018060...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018060...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018060...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018060...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018060...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018060...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018060...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018061...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018061...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018061...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018061...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018061...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018061...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018061...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018061...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018061...


In [182]:
# use code below to locate folders of individual animals:
info.loc['folder','glut1_f8wks1']

'/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/20180611_085108_Glut1_2_10'

## Main loop

Runs the selected analyses (modules) for each individual animal.

- To turn a certain analysis on or off: comment/uncomment the line with *#*
- to run the loop only on a single/subset of animals: index *info.columns*, e.g. info.columns[1:2] would run only the 2nd animal

In [3]:
# for animal in ['glut1_f8wks1']:  # to run only one or subset of animals, enter here and comment out line below.
for animal in info.columns:   
    print('animal: ',animal)
    folders['animal']=info.loc['folder',animal]
    inputs = dict(folders=folders, animal=animal, info=info.to_json())
    
#     h.run_module(folders, 'Angiography', inputs)  # ~2-3 min

    h.run_module(folders, 'BolusTracking', inputs, overwrite = True) # ~20 min

#     h.run_module(folders, 'Diamox', inputs, overwrite = True) # ~2-3 min

print('All complete!')

animal:  glut1_male1
running Diamox analysis...


Input Notebook:  /BigHDD/MR_DATA/GithubRepositories/CalciumfMRI/fMRI_notebooks/Diamox.ipynb
Output Notebook: /home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/20180604_101508_Glut1_2_2/Diamox_output.ipynb
100%|██████████| 18/18 [01:57<00:00,  6.54s/it]


animal:  glut1_male2
running Diamox analysis...


Input Notebook:  /BigHDD/MR_DATA/GithubRepositories/CalciumfMRI/fMRI_notebooks/Diamox.ipynb
Output Notebook: /home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/20180604_135437_Glut1_2_3/Diamox_output.ipynb
 44%|████▍     | 8/18 [00:02<00:02,  3.99it/s]

KeyboardInterrupt: 

## Collecting outputs for group analysis

copies only relevant files of each animal in group folder

In [189]:
folders['group'] = h.folder_check_create(folders['main'], 'Groupstats')


for animal in info.columns:   
    folders['animal']=info.loc['folder',animal]

#     info = h.copy_notebook_outputs(folders, animal, 'Angiography', info)

#     info, folders = h.copy_notebook_outputs(folders, animal, 'BolusTracking', info)
    
    info, folders = h.copy_notebook_outputs(folders, animal, 'Diamox', info)
    
    template_file = os.path.join(folders['main'], 'Templates', 'BolusTracking.nii')    
    output = os.path.join(folders['group'], 'BolusTracking', 'Template.nii')
    shutil.copy(template_file, output)
    output = os.path.join(folders['group'], 'Diamox', 'Template.nii') # same scan settings as BolusTracking
    shutil.copy(template_file, output)
    
info

,glut1_male1,glut1_male2,glut1_f10wks1,glut1_f10wks2,glut1_f10wks3,glut1_f10wks4,glut1_f10wks5,glut1_f10wks6,glut1_f8wks1,glut1_f8wks2,glut1_f8wks3,glut1_f8wks4,glut1_f8wks5,glut1_f8wks6,glut1_f8wks7,glut1_f6wks2,glut1_f6wks1
scan,,,,,,,,,,,,,,,,,
genotype,1,1,1,2,1,2,2,1,1,1,2,2,2,2,1,1,1
Gd DCE,6,8,6,5,5,8,5,5,7,5,6,6,8,8,6,5,5
Angio pre,8,9,7,6,6,9,6,6,8,6,9,8,9,9,8,6,6
Angio post,12,12,11,9,9,13,10,11,12,11,12,12,16,13,12,9,10
Endorem DCE,11,11,10,8,8,12,9,10,10,10,11,11,15,12,11,8,9
Diamox,15,14,13,11,13,15,12,13,15,13,16,17,20,16,14,11,13
folder,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018060...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018060...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018060...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018060...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018060...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018060...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018060...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018060...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018061...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018061...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018061...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018061...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018061...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018061...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018061...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018061...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018061...
CBV_steady,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CBV_bolus,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Create average & statistical maps

### Bolus Tracking

In [168]:
map_names = ['CBF_map', 'CBV_bolus_map', 'MTT_map']   # prefixes of the output files

for map_type in map_names:
    group1, group2 = h.select_for_group_average(folders['group_BolusTracking'],
                             info, map_type,
                             exclude='male',  #<-- excludes animals that contain this string in animal name
                             row = 'genotype')
    h.ttest_2groups(folders['group_BolusTracking'], map_type, group1, group2, blur=0.3)

created /home/aic/BigHDD/MR_DATA/GLUT1/Groupstats/BolusTracking/meanCBF_map_Ttest.nii
created /home/aic/BigHDD/MR_DATA/GLUT1/Groupstats/BolusTracking/meanCBV_bolus_map_Ttest.nii
created /home/aic/BigHDD/MR_DATA/GLUT1/Groupstats/BolusTracking/meanMTT_map_Ttest.nii


### Diamox

In [191]:
map_names = ['diamox_ttp', 'diamox_slope']

for map_type in map_names:
    group1, group2 = h.select_for_group_average(folders['group_Diamox'],
                             info, map_type,
                             exclude='male',  #<-- excludes animals that contain this string in animal name
                             row = 'genotype')
    h.ttest_2groups(folders['group_Diamox'], map_type, group1, group2, blur=0.3)

created /home/aic/BigHDD/MR_DATA/GLUT1/Groupstats/Diamox/meandiamox_ttp_Ttest.nii
created /home/aic/BigHDD/MR_DATA/GLUT1/Groupstats/Diamox/meandiamox_slope_Ttest.nii
